In [ ]:
!pip install -q chromadb sentence-transformers --quiet

In [5]:
import chromadb
from chromadb.config import Settings
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from datetime import datetime
import time
import os

In [6]:
from os.path import exists
persist_dir = "./chroma_time_window_db"
if not os.path.exists(persist_dir):
  os.makedirs(persist_dir, exist_ok=True)

In [7]:
client= chromadb.PersistentClient(path = persist_dir)

In [ ]:
collection_name = "time_memory"
collection = client.get_or_create_collection(name=collection_name)


In [26]:
def store_message(user_text):
  timestamp = time.time()
  collection.add(documents=[user_text], metadatas=[{ "timestamp": timestamp }],
                 ids=[f"id-{timestamp}"]
                 )
  print("Agent: Noted.")

In [27]:
def show_recent_messages(minutes_back=1):
  since_time = time.time()-(minutes_back*60)
  results = collection.query(
      query_texts=["recent memory"],
      n_results=10,
      where={"timestamp": {"$gte":since_time}}
  )
  print(f"Last {minutes_back} minutes of memory:")
  for doc in results["documents"][0]:
    print(f" - {doc}")

In [23]:
def clear_memory():
  client.delete_collection(name=collection_name)
  print("Agent: All memory has been cleared.")

In [ ]:
# Interactive prompt
print("Time-Windowed Memory Agent (type 'show' to view last 1 minutes, 'clear' to erase memory, 'exit' to quit)")
while True:
    user_input = input("You: ").strip()
    if user_input.lower() == "exit":
        print("Agent: Goodbye!")
        break
    elif user_input.lower() == "clear":
        clear_memory()
    elif user_input.lower() == "show":
        show_recent_messages(minutes_back=1)
    else:
        store_message(user_input)

Time-Windowed Memory Agent (type 'show' to view last 1 minutes, 'clear' to erase memory, 'exit' to quit)
You: clear
Agent: All memory has been cleared.
